In [12]:
#Load necessary libraries
import mne
import os
import numpy as np
import pandas as pd
import scipy.io as sio
from scipy import stats
from fldtrp2mne_calc import fldtrp2mne_calc
from calc_classification import calc_classification
from calc_regression import calc_regression
from calc_decoding_cfg import (result_path)

cwd = os.path.dirname(os.path.abspath(__file__))
os.chdir(cwd)


NameError: name '__file__' is not defined

In [34]:
info

,run,operand1,operator,operand2,preResult,delay,corrResult,deviant,absdeviant,rt,respSide
0,1,5,1,2,6,1,7,-1,1,996,1
1,1,3,-1,1,3,0,2,1,1,1085,1
2,1,5,0,33,1,0,5,-4,4,1126,1
3,1,5,1,2,6,1,7,-1,1,1003,1
4,1,6,-1,2,7,1,4,3,3,1162,1
5,1,4,-1,2,1,0,2,-1,1,1054,1
6,1,6,1,3,5,0,9,-4,4,1116,1
7,1,4,0,33,4,1,4,0,0,1207,-1
8,1,5,0,33,1,0,5,-4,4,997,1
9,1,6,-1,0,2,0,6,-4,4,1160,1


In [32]:
from calc_decoding_cfg import (data_path, baseline, downsampling)
Condition = ['op1', 'op1']
Subject = 's01'

#Decoding
trainset = Condition[0] # and then this will be already known when calling the function
testset = Condition[1]

params = {'baseline': baseline, 'downsampling': downsampling,
'classification': Condition, 'trainset': trainset, 
'testset': testset, 'mode': mode}

fname = data_path + '/' + Subject + '_calc.mat'
epoch_calc,info_calc = fldtrp2mne_calc(fname, 'data', 'calc')
epoch_vsa,info_vsa = fldtrp2mne_calc(fname, 'data', 'vsa')
epoch_calc.apply_baseline(baseline)
epoch_vsa.apply_baseline(baseline)

#Downsample data if needed
if downsampling > 0:
	epoch_calc.decimate(downsampling)
    epoch_vsa.decimate(downsampling)

#Select data to use.

if trainset[-1] == testset[-1]: #Check whether we are training and testing on the same data
    mode = 'cross-validation'
    if trainset == 'op1':
        X_train = epoch_calc
        y_train = np.array(info_calc['operand1'])
        y_train = y_train.astype(numpy.float64)
        X_test = X_train
        y_test = y_train
    elif: trainset == 'op2':
            X_train = epoch_calc
            y_train = np.array(info_calc['operand2'])
            y_train = y_train.astype(numpy.float64)
            X_test = X_train
            y_test = y_train
    elif: trainset == 'pres':
            X_train = epoch_calc[(info_calc['preResult'] >= 3) & (info_calc['preResult'] <= 6)]
            y_train = np.array(info['preResult'])
            y_train = y_train.astype(numpy.float64)
            X_test = X_train
            y_test = y_train
    elif: trainset == 'cres':
            X_train = epoch_calc[(info_calc['corrResult'] >= 3) & (info_calc['preResult'] <= 6)]
            y_train = np.array(info['corrResult'])
            y_train = y_train.astype(numpy.float64)
            X_test = X_train
            y_test = y_train
else:
    mode = 'mean-prediction'

gat_r, score_r, diagonal_r = calc_regression(X_train, y_train, X_test, y_test, params)
gat_c, score_c, diagonal_c = calc_classification(X_train, y_train, X_test, y_test, params)

return params, epoch_calc.times, epoch_vsa.times, gat_r, score_r, diagonal_r, gat_c, score_c, diagonal_c


params, times_calc, times_vsa, gat_r, score_r, diagonal_r, gat_c, score_c, diagonal_c = calc_prepDec_wTask_CR(wkdir, Condition, Subject)

results = {'params': params, 'times_calc': times_calc, 'gat_r': gat_r, 'score_r': score_r, 
'gat_c': gat_c, 'score_c': score_c, 'diagonal_c': diagonal_c}

#Save data
fname = result_path + '/individual_results/' + Subject + '_' + trainset + '_' + testset + '-results'
np.save(fname, results)

Applying baseline correction ... (mode: mean)


In [35]:
result_path

'/neurospin/meg/meg_tmp/Calculation_Pedro_2014/data/decoding'

In [52]:
results = {'params': params, 'a': a}

In [54]:
results['a']

array([ 1,  1, 10])